In [ ]:
# needed to load files in google colab
from google.colab import drive
drive.mount('/content/drive')

import os
localsource='/content/drive/MyDrive/Predicting_3D_Magnetic_Topological_Insulators_and_Semimetals/ExcelFiles/'

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install mp_api

In [ ]:
# !pip install kogger

In [ ]:
"""
Download materials from materialsproject.org

"""
from mp_api.client import MPRester
import os.path as osp
import pandas as pd
# import kogger
from tqdm import tqdm

In [ ]:
def read_material_ids(data_path):
    """
    Parameters
    ----------
    data_path: file path of sample.csv

    Returns
    -------
    material_ids: list

    """
    df = pd.read_csv(data_path, header=None)
#    kogger.info('Read {} samples from {}'.format(len(df), data_path))
    material_ids = df[1].tolist()
    return material_ids

api_key = "Ead63fZDGQYh6UElvtbcEpYTcZk6WTjZ"

# label = 1 for label1.csv, 0 for label0.csv, -1 for candidate.csv
label = -1

data_path = localsource+'data/candidate.csv'
save_dir = localsource+'candidate'
save_dir2 = localsource+'data'

material_ids = read_material_ids(data_path)

with MPRester(api_key) as mpr:
    pull=100
    for i in range(0,1+int(len(material_ids)/pull)):
        material_subset=material_ids[i*pull:(i+1)*pull]
        if os.path.isfile(save_dir+'/'+material_ids[i*pull]+'.cif')==False: #these 100 materials have not been saved yet.
            docs = mpr.materials.bonds.search(material_ids=material_subset, fields=['material_id', 'structure_graph'])
            length = len(docs)
            for idx in tqdm(range(length)):
                struc = docs[idx].structure_graph.structure
                id = docs[idx].material_id
                # save file,  'data/mp-22862.cif'
                struc.to(filename=osp.join(save_dir, '{}.cif'.format(id)))

    # append to 'data/id_prop_{}.csv'
    labels = [label] * len(material_ids)
    df = pd.DataFrame(list(zip(material_ids, labels)))
    path = osp.join(save_dir2, 'id_prop_{}.csv'.format(label))
    # kogger.info('Appending to {}'.format(path))
    df.to_csv(path, header=None, index=None, mode='a')